pca_tfidf_150features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/tfidf_withstopwordchangelevels_150_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,0,-0.904773,-0.619597,-0.122628,-0.072776,-1.130577,-0.071786,0.191583,-0.419179,-0.027590,...,-0.205305,-3.613914,3.334850,-2.655560,2.457335,0.034222,0.187660,1.123405,-1.288926,6.477600
1,1,-0.637927,-0.476528,-0.163228,-0.120705,-1.250071,-0.314732,0.254706,-0.269941,-0.153382,...,-0.401731,-0.942918,1.348969,-0.062138,-0.224317,0.342368,0.150814,1.018795,0.271233,-0.130134
2,2,-0.784083,-0.495026,-0.119506,-0.038586,-0.660604,-0.010356,0.112481,-0.139476,-0.069200,...,-0.046741,-0.108923,0.403727,0.311409,-0.028207,-0.186569,-0.365606,0.196748,-0.302130,0.101195
3,3,-0.735979,-0.440759,-0.102810,-0.049273,-0.749807,-0.121789,0.069119,-0.112774,-0.075759,...,-0.099386,-0.322988,0.248142,0.731263,-0.250659,-0.145451,-0.038558,0.839371,-0.012263,0.127685
4,4,-0.775996,-0.486196,-0.116605,-0.063713,-0.937287,-0.158065,0.097456,-0.122350,-0.101949,...,-0.216282,-0.462366,0.273024,0.945268,-0.180579,0.013634,0.247404,0.809886,0.086290,0.421667


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.904773,-0.619597,-0.122628,-0.072776,-1.130577,-0.071786,0.191583,-0.419179,-0.027590,-0.090389,...,-0.205305,-3.613914,3.334850,-2.655560,2.457335,0.034222,0.187660,1.123405,-1.288926,6.477600
1,-0.637927,-0.476528,-0.163228,-0.120705,-1.250071,-0.314732,0.254706,-0.269941,-0.153382,0.179296,...,-0.401731,-0.942918,1.348969,-0.062138,-0.224317,0.342368,0.150814,1.018795,0.271233,-0.130134
2,-0.784083,-0.495026,-0.119506,-0.038586,-0.660604,-0.010356,0.112481,-0.139476,-0.069200,0.078934,...,-0.046741,-0.108923,0.403727,0.311409,-0.028207,-0.186569,-0.365606,0.196748,-0.302130,0.101195
3,-0.735979,-0.440759,-0.102810,-0.049273,-0.749807,-0.121789,0.069119,-0.112774,-0.075759,0.066954,...,-0.099386,-0.322988,0.248142,0.731263,-0.250659,-0.145451,-0.038558,0.839371,-0.012263,0.127685
4,-0.775996,-0.486196,-0.116605,-0.063713,-0.937287,-0.158065,0.097456,-0.122350,-0.101949,0.090910,...,-0.216282,-0.462366,0.273024,0.945268,-0.180579,0.013634,0.247404,0.809886,0.086290,0.421667


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 150), (524, 150))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [16]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [17]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3482952456314202


**Decision Tree**

In [18]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [19]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [20]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [21]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [22]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [23]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.529508,0.529508,0.523417,0.51675,0.364504,0.364504,0.342071,0.348449


**Random Forest**

In [24]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [25]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [26]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [27]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9972    0.9945    0.9958       361
           2     0.9964    0.9946    0.9955       553
           3     0.9870    0.9935    0.9902       306

    accuracy                         0.9943      1220
   macro avg     0.9935    0.9942    0.9938      1220
weighted avg     0.9943    0.9943    0.9943      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2078    0.1019    0.1368       157
           2     0.4635    0.7764    0.5804       237
           3     0.2600    0.1000    0.1444       130

    accuracy                         0.4065       524
   macro avg     0.3104    0.3261    0.2872       524
weighted avg     0.3364    0.4065    0.3393       524



In [28]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.529508,0.529508,0.523417,0.516750,0.364504,0.364504,0.342071,0.348449
1,RandomForest,0.994262,0.994262,0.994278,0.994267,0.406489,0.406489,0.336388,0.339337


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6033    0.4044    0.4842       361
           2     0.5270    0.7758    0.6277       553
           3     0.5793    0.3105    0.4043       306

    accuracy                         0.5492      1220
   macro avg     0.5699    0.4969    0.5054      1220
weighted avg     0.5627    0.5492    0.5292      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2600    0.1656    0.2023       157
           2     0.4444    0.6582    0.5306       237
           3     0.2740    0.1538    0.1970       130

    accuracy                         0.3855       524
   macro avg     0.3261    0.3259    0.3100       524
weighted avg     0.3469    0.3855    0.3495       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.574590,0.574590,0.584638,0.548821,0.393130,0.393130,0.353836,0.359223
1,RandomForest,0.994262,0.994262,0.994278,0.994267,0.374046,0.374046,0.330165,0.329957
2,Adaboost,0.549180,0.549180,0.562701,0.529185,0.385496,0.385496,0.346889,0.349499


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9704    0.9086    0.9385       361
           2     0.8934    0.9855    0.9372       553
           3     0.9853    0.8758    0.9273       306

    accuracy                         0.9352      1220
   macro avg     0.9497    0.9233    0.9344      1220
weighted avg     0.9393    0.9352    0.9351      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3214    0.2293    0.2677       157
           2     0.4589    0.6835    0.5492       237
           3     0.2881    0.1308    0.1799       130

    accuracy                         0.4103       524
   macro avg     0.3562    0.3479    0.3322       524
weighted avg     0.3754    0.4103    0.3732       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.574590,0.574590,0.584638,0.548821,0.393130,0.393130,0.353836,0.359223
1,RandomForest,0.994262,0.994262,0.994278,0.994267,0.374046,0.374046,0.330165,0.329957
2,Adaboost,0.549180,0.549180,0.562701,0.529185,0.385496,0.385496,0.346889,0.349499
3,GBM,0.935246,0.935246,0.939257,0.935120,0.410305,0.410305,0.375356,0.373202


**SVM Model**

In [29]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [30]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [31]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [32]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5106    0.5983    0.5510       361
           2     0.6827    0.5136    0.5862       553
           3     0.4751    0.5915    0.5269       306

    accuracy                         0.5582      1220
   macro avg     0.5561    0.5678    0.5547      1220
weighted avg     0.5797    0.5582    0.5609      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3198    0.4013    0.3559       157
           2     0.4533    0.2869    0.3514       237
           3     0.2316    0.3154    0.2671       130

    accuracy                         0.3282       524
   macro avg     0.3349    0.3345    0.3248       524
weighted avg     0.3583    0.3282    0.3319       524



In [33]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.529508,0.529508,0.523417,0.516750,0.364504,0.364504,0.342071,0.348449
1,RandomForest,0.994262,0.994262,0.994278,0.994267,0.406489,0.406489,0.336388,0.339337
2,SVM,0.558197,0.558197,0.579704,0.560911,0.328244,0.328244,0.358323,0.331854


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4879    0.4460    0.4660       361
           2     0.5630    0.7269    0.6346       553
           3     0.4830    0.2778    0.3527       306

    accuracy                         0.5311      1220
   macro avg     0.5113    0.4836    0.4844      1220
weighted avg     0.5207    0.5311    0.5140      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2848    0.2994    0.2919       157
           2     0.4266    0.5274    0.4717       237
           3     0.2273    0.1154    0.1531       130

    accuracy                         0.3569       524
   macro avg     0.3129    0.3141    0.3056       524
weighted avg     0.3347    0.3569    0.3388       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.574590,0.574590,0.584638,0.548821,0.393130,0.393130,0.353836,0.359223
1,RandomForest,0.994262,0.994262,0.994278,0.994267,0.374046,0.374046,0.330165,0.329957
2,Adaboost,0.549180,0.549180,0.562701,0.529185,0.385496,0.385496,0.346889,0.349499
3,GBM,0.935246,0.935246,0.939257,0.935120,0.410305,0.410305,0.375356,0.373202
4,SVM,0.589344,0.589344,0.685080,0.546186,0.410305,0.410305,0.351188,0.348380
5,KNN,0.531148,0.531148,0.520706,0.513988,0.356870,0.356870,0.334687,0.338784


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3842    0.2022    0.2650       361
           2     0.7083    0.0922    0.1632       553
           3     0.2672    0.8366    0.4051       306

    accuracy                         0.3115      1220
   macro avg     0.4533    0.3770    0.2777      1220
weighted avg     0.5018    0.3115    0.2540      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3038    0.1529    0.2034       157
           2     0.2800    0.0295    0.0534       237
           3     0.2429    0.7846    0.3709       130

    accuracy                         0.2538       524
   macro avg     0.2756    0.3223    0.2092       524
weighted avg     0.2779    0.2538    0.1771       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.574590,0.574590,0.584638,0.548821,0.393130,0.393130,0.353836,0.359223
1,RandomForest,0.994262,0.994262,0.994278,0.994267,0.374046,0.374046,0.330165,0.329957
2,Adaboost,0.549180,0.549180,0.562701,0.529185,0.385496,0.385496,0.346889,0.349499
3,GBM,0.935246,0.935246,0.939257,0.935120,0.410305,0.410305,0.375356,0.373202
4,SVM,0.589344,0.589344,0.685080,0.546186,0.410305,0.410305,0.351188,0.348380
5,KNN,0.531148,0.531148,0.520706,0.513988,0.356870,0.356870,0.334687,0.338784
6,GNB,0.311475,0.311475,0.501786,0.253979,0.253817,0.253817,0.277915,0.177127
